In [ ]:
#include <vector>
#include <map>
#include <algorithm>
#include <cmath>
#include <iostream>
using namespace std;

map<int, double> get_frequency_dict(vector<int> numbers, int n) {
    map<int, double> frequency_dict;
    for (int num : numbers) {
        frequency_dict[num]++;
    }
    for (auto& pair : frequency_dict) {
        pair.second /= n;
    }
    return frequency_dict;
}

double get_expectation_2(vector<int> numbers, int n) {
    double expectation = 0;
    map<int, double> frequency_dict = get_frequency_dict(numbers, n);
    int unique = frequency_dict.size();

    vector<int> numbers_unique(unique);
    vector<double> probas_unique(unique);
    int index = 0;
    for (auto& pair : frequency_dict) {
        numbers_unique[index] = pair.first;
        probas_unique[index] = pair.second;
        index++;
    }

    vector<double> cdf_z(unique + 1, 0);
    vector<double> cdf_x1(unique + 1, 0);
    for (int k = 0; k < unique + 1; k++) {
        for (int j = 0; j < k; j++) {
            cdf_x1[k] += probas_unique[j];
        }
        cdf_x1[k] = 1 - pow(1 - cdf_x1[k], 4);
    }
    for (int j = 0; j < unique; j++) {
        expectation += numbers_unique[j] * (cdf_x1[j + 1] - cdf_x1[j]);
    }
    return expectation;
}

double get_expectation_1(vector<int> numbers, int n) {
    double expectation = 0;
    for (int number : numbers) {
        expectation += number;
    }
    expectation /= n;
    return expectation;
}

double main_function(vector<int> numbers, int n) {
    double exp_2 = get_expectation_2(numbers, n);
    double exp_1 = get_expectation_1(numbers, n);
    return exp_1 * 4 - exp_2;
}

pair<int, double> trial_simulation(vector<int> sizes, vector<vector<int>> cards) {
    vector<double> max_means(2, 0);
    vector<int> max_mean_indices(2, -1);

    for (int i = 0; i < sizes.size(); i++) {
        double mean = 0;
        for (int j = 0; j < sizes[i]; j++) {
            mean += cards[i][j];
        }
        mean /= sizes[i];

        if (mean > max_means[0]) {
            max_means[1] = max_means[0];
            max_mean_indices[1] = max_mean_indices[0];
            max_means[0] = mean;
            max_mean_indices[0] = i;
        } else if (mean > max_means[1]) {
            max_means[1] = mean;
            max_mean_indices[1] = i;
        }
    }

    double exp_max1 = main_function(cards[max_mean_indices[0]], sizes[max_mean_indices[0]]);
    double exp_max2 = main_function(cards[max_mean_indices[1]], sizes[max_mean_indices[1]]);

    if (exp_max1 > exp_max2) {
        return make_pair(max_mean_indices[0] + 1, exp_max1);
    } else {
        return make_pair(max_mean_indices[1] + 1, exp_max2);
    }
}


int main() {
    int N;
    cin >> N;
    vector<int> sizes(N, 0);
    vector<vector<int>> cards(N);
    for (int n = 0; n < N; n++) {
        cin >> sizes[n];
        cards[n].resize(sizes[n]);
        for (int i = 0; i < sizes[n]; i++) {
            cin >> cards[n][i];
        }
    }
    pair<int, double> result = trial_simulation(sizes, cards);
    cout << result.first << " " << result.second << endl;
    return 0;
}

In [ ]:
#include <iostream>
#include <vector>
#include <sstream>
using namespace std;

const long long mod = 1000000007;

vector<vector<long long>> matrix_mul_mod(vector<vector<long long>> matrix1, vector<vector<long long>> matrix2, long long modulus) {
    vector<vector<long long>> result(matrix1.size(), vector<long long>(matrix2[0].size(), 0));
    for (int i = 0; i < matrix1.size(); i++) {
        for (int j = 0; j < matrix2[0].size(); j++) {
            for (int k = 0; k < matrix1[0].size(); k++) {
                result[i][j] = (result[i][j] + (matrix1[i][k] * matrix2[k][j]) % modulus) % modulus;
            }
        }
    }
    return result;
}

vector<vector<long long>> matrix_power_mod(vector<vector<long long>> matrix, long long power, long long modulus) {
    vector<vector<long long>> result(matrix.size(), vector<long long>(matrix.size(), 0));
    for (int i = 0; i < matrix.size(); i++) {
        result[i][i] = 1;
    }

    while (power > 0) {
        if (power % 2 == 1) {
            result = matrix_mul_mod(result, matrix, modulus);
        }
        matrix = matrix_mul_mod(matrix, matrix, modulus);
        power = power / 2;
    }
    return result;
}

string get_coords(long long circles, vector<long long> coords) {
    vector<vector<long long>> B = matrix_power_mod({{0, 1, 1}, {0, 1, 2}, {0, 2, 3}}, circles, mod);
    vector<long long> result(3, 0);
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            result[i] = (result[i] + (B[i][j] * coords[j]) % mod) % mod;
        }
    }
    ostringstream out;
    for (int i = 0; i < result.size(); i++) {
        out << result[i];
        if (i < result.size() - 1) {
            out << " ";
        }
    }
    return out.str();
}

int main() {
    int N;
    cin >> N;
    for (int n = 0; n < N; n++) {
        long long circles[2], coords[3];
        for (int j = 0; j < 2; j++) {
            cin >> circles[j];
        }
        for (int i = 0; i < 3; i++) {
            cin >> coords[i];
        }
        cout << get_coords(circles[1], vector<long long>(coords, coords + 3)) << endl;
    }
    return 0;
}
    

In [ ]:
# not optimized for fast computation. There is an upper bound for expectation
# of an array or mean of array that will give the maximum to our defined expectation
# however Yandex didn't like such approach and still didn't accept it
# so I cheated a little and instead of computing the code on only 1 array from
# all inputs with maximum mean, I computed it for 2 with biggest mean and it worked.
# Should notice the incompetence of competions' administration as they didn't give extra time
# for python. Instead solution can be only possible to be written in C++ due to time limits.

from collections import Counter
def get_expectation_m(numbers, n):
    expectation = 0
    frequency_dict = Counter(numbers)
    for key in frequency_dict:
        frequency_dict[key] /= n
    numbers, probas = list(frequency_dict.keys()), list(frequency_dict.values())
    unique = len(probas)
    cdf_x1 = [sum(probas[:k]) for k in range(unique + 1)]
    cdf_z = [(1 - (1-cdf_x1[s])**4) for s in range(unique + 1)]
    expectation = sum(numbers[j] * (cdf_z[j+1] - cdf_z[j]) for j in range(unique))
    return expectation

def get_expectation(numbers, n):
    proba = 1/n
    expectation = 0
    for i in range(n):
        expectation += numbers[i]*proba
    return expectation

def main(numbers, n):
    exp_m = get_expectation_m(numbers, n)
    exp_x1 = get_expectation(numbers, n)
    return exp_x1*4 - exp_m

N = int(input()) 
sizes = [0]*N
cards = [0]*N
for n in range(N):
    sizes[n] = int(input())
    cards[n] = list(map(int, input().split(' ')))
test = zip(sizes, cards)
trials = [0] * N
exp_maximums = [0] * N
for trial, (size, card) in enumerate(test):
    exp_maximums[trial] = round(main(card, size), 4)
    trials[trial] = trial+1
exp_max = max(exp_maximums)
index = exp_maximums.index(exp_max)
print(trials[index], exp_max)

In [ ]:
# well that was tricky as I have never seen computational stability problems in
# algorithm section so I didn't know how to approach and was forced to google
# other peoples' solution to computation of large powers of a matrix.

import numpy as np
A = np.array([[0, 1, 1],
             [0, 1, 2],
             [0, 2, 3]])
mod = 1000000007

def matrix_power_mod(matrix, power, modulus):
    result = [[int(row == column) for column in range(len(matrix))] for row in range(len(matrix))]
    while power > 0:
        if power % 2 == 1:
            result = matrix_mul_mod(result, matrix, modulus)
        matrix = matrix_mul_mod(matrix, matrix, modulus)
        power = power // 2
    return result

def matrix_mul_mod(matrix1, matrix2, modulus):
    return [[sum((matrix1[row][k] * matrix2[k][column]) % modulus for k in range(len(matrix1[0]))) for column in range(len(matrix2[0]))] for row in range(len(matrix1))]

def get_coords(circles, coords):
    coords = np.array(coords).T
    B = matrix_power_mod(A, circles, mod)
    result = (B @ coords) % mod
    return ' '.join(map(str, result.tolist()))

N = int(input()) 
coords = [0]*N
circles = [0]*N
for n in range(N):
    print(get_coords(list(map(int, input().split(' ')))[1], list(map(int, input().split(' ')))))
